# Import packages

In [1]:
import math 
import numpy as np 

import pandas as pd 
import os 
import csv 

from tqdm import tqdm 

import torch 
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader, random_split 

from torch.utils.tensorboard import SummaryWriter

# Some utility function

In [2]:
def same_seed(seed):
    '''Fixes random number generator seeds for reproducibility'''
    torch.backends.cudnn.deterministic = True 
    torch.backends.cudnn.benchmark = False 
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


In [3]:
def train_valid_split(data_set, valid_ratio, seed):
    '''Split provided training data into training set and 
    validation set'''
    valid_set_size = int(valid_ratio * len(data_set))
    train_set_size = len(data_set) - valid_set_size 
    train_set, valid_set = random_split(data_set, 
        [train_set_size, valid_set_size])
    return np.array(train_set), np.array(valid_set)

In [4]:
def predict(test_loader, model, device):
    model.eval() # set the model for evaluation 
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)
        with torch.no_grad():
            pred = model(x)
            preds.append(pred.detach().cpu())
    
    preds = torch.cat(preds, dim=0).numpy()
    return preds

# Dataset

In [5]:
class COVID19Dataset(Dataset):
    '''
    x: features
    y: targets, if none, do predictions
    '''
    def __init__(self, x, y=None):
        if y is None:
            self.y = y  
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

# Neural Network Model 



In [6]:
class My_Model(nn.Module):

    def __init__(self, input_dim):
        super(My_Model, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1) # (B, 1) - > (B)
        return x

# Feature Selection

In [7]:
def select_feat(train_data, valid_data, test_data, select_all=True):
    '''Select useful features to perform regression'''
    y_train, y_valid = train_data[:, -1], valid_data[:, -1]
    raw_x_train, raw_x_valid, raw_x_test = train_data[:, :-1], \
        valid_data[:, :-1], test_data
    
    if select_all:
        feat_idx = list(range(raw_x_train.shape[1]))
    else:
        feat_idx = [0, 1, 2, 3, 4]
    
    return raw_x_train[:, feat_idx], raw_x_valid[:, feat_idx], \
        raw_x_test[:, feat_idx], y_train, y_valid

# Training loop

In [8]:
def trainer(train_loader, valid_loader, model, config, device):
    criterion = nn.MSELoss(reduction="mean") # define loss function 

    # define the optimization algoritum
    optimizer = torch.optim.SGD(model.parameters(), 
        lr=config['learning_rate'], momentum=0.9)
    writer = SummaryWriter() # writer of tensorboard

    if not os.path.isdir('./models'):
        os.mkdir('./models') # create directory of saving models 

    n_epochs, best_loss, step, early_stop_count = \
        config['n_epochs'], math.inf, 0, 0 

    for epoch in range(n_epochs):
        model.train() # set the model to train mode
        loss_record = []

        # tqdm is a package to visualize the training progress
        train_pbar = tqdm(train_loader, position=0, leave=True)

        for x, y in train_pbar:
            optimizer.zero_grad() # set the gradient to zero
            x, y = x.to(device), y.to(device) # move data to device
            pred = model(x)
            loss = criterion(pred, y)
            loss.backward() # compute gradient (backpropagation)
            optimizer.step() # update parameter
            step += 1
            loss_record.append(loss.detach().item())

            # display current epoch number and loss on tqdm progress bar
            train_pbar.set_description(f'epoch [{epoch+1}/{n_epochs}]')
            train_pbar.set_postfix({'loss': loss.detach().item()})
        
        mean_train_loss = sum(loss_record)/len(loss_record)
        writer.add_scalar('Loss/train', mean_train_loss, step)

        model.eval() # set the model to evaluation mode
        loss_record = [] 
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())

        mean_valid_loss = sum(loss_record) / len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, \
            Valid loss: {mean_valid_loss:.4f}')
        writer.add_scalar('Loss/valid', mean_valid_loss, step)

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # save the best model
            print('Saving model with loss {:.3f}'.format(best_loss))
            early_stop_count = 0
        else:
            early_stop_count += 1 

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session')
            return 

# Configurations 

config containss the hyper-parameter for training and the path to save the model

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'seed': 5201314, # defined your seed number
    'select_all': True, # whether to use all features
    'valid_ratio': 0.2, # validation_size = train_size * valid_ratio
    'n_epochs': 3000, # number of epochs
    'batch_size': 256, 
    'learning_rate': 1e-5,
    'early_stop': 400, # if model has not improved for some epochs
    'save_path': './models/model.ckpt'
}

# Data loader

In [10]:
# set seed for reproducibility
same_seed(config['seed'])

# train_data size: 2699 * 118 (id + 37 states + 16 features * 5 days)
# test_data size: 1078 * 177 (without last day's positive rate)
train_data, test_data = pd.read_csv('./datasets/covid.train.csv').values, \
    pd.read_csv('./datasets/covid.test.csv').values 

train_data, test_data

(array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         8.62711740e+00, 3.73295118e+01, 7.45615380e+00],
        [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         9.95433280e+00, 3.25088806e+01, 8.01095700e+00],
        [2.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         7.95613890e+00, 3.67455876e+01, 2.90697740e+00],
        ...,
        [2.69700000e+03, 0.00000000e+00, 0.00000000e+00, ...,
         1.28882747e+01, 4.24524509e+01, 1.36182563e+01],
        [2.69800000e+03, 0.00000000e+00, 0.00000000e+00, ...,
         1.00969392e+01, 2.96688018e+01, 9.51258460e+00],
        [2.69900000e+03, 0.00000000e+00, 0.00000000e+00, ...,
         9.61811390e+00, 3.77441952e+01, 1.10694227e+01]]),
 array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.24525703e+01, 9.40002440e+00, 3.05508734e+01],
        [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.17475281e+01, 7.43860580e+00, 3.53648714e+01],
        [2.00000000e+00, 

In [11]:
train_data, valid_data = train_valid_split(train_data,
    config['valid_ratio'], config['seed'])

# print the data size
print(f"""train_data size: {train_data.shape}
valid_data size: {valid_data.shape}
test_data size: {test_data.shape}
""")

train_data size: (2160, 118)
valid_data size: (539, 118)
test_data size: (1078, 117)



In [12]:
# select features 
x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data, 
    valid_data, test_data, config['select_all'])

# print the number of features
print(f'number of features: {x_train.shape[1]}')

number of features: 117


In [13]:
train_dataset, valid_dataset, test_dataset = \
        COVID19Dataset(x_train, y_train), \
        COVID19Dataset(x_valid, y_valid), \
        COVID19Dataset(x_test)

# pytorch data loader loads pytorch dataset into batches 
train_loader = DataLoader(
    train_dataset, batch_size=config['batch_size'])
valid_loader = DataLoader(
    valid_dataset, batch_size=config['batch_size'],
    shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, 
    batch_size=config['batch_size'], shuffle=False, pin_memory=True)

# Start Training

In [14]:
# put model and data on the same computation device
model = My_Model(input_dim=x_train.shape[1]).to(device)

trainer(train_loader, valid_loader, model, config, device)

2023-02-25 00:45:03.066342: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-25 00:45:04.151593: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-25 00:45:09.045521: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ros/humble/opt/rviz_ogre_vendor/lib:/opt/ros/humble/lib/x86_64-linux-gnu:/opt/ros/

Epoch [1/3000]: Train loss: 76.6408,             Valid loss: 62.9660
Saving model with loss 62.966


epoch [2/3000]: 100%|██████████| 9/9 [00:00<00:00, 76.41it/s, loss=33.3]


Epoch [2/3000]: Train loss: 45.3118,             Valid loss: 35.0970
Saving model with loss 35.097


epoch [3/3000]: 100%|██████████| 9/9 [00:00<00:00, 77.45it/s, loss=31.7]


...

Epoch [1771/3000]: Train loss: 1.9264,             Valid loss: 2.8830


epoch [1772/3000]: 100%|██████████| 9/9 [00:00<00:00, 82.86it/s, loss=1.17]


Epoch [1772/3000]: Train loss: 1.9379,             Valid loss: 2.6574


epoch [1773/3000]: 100%|██████████| 9/9 [00:00<00:00, 95.12it/s, loss=1.16]


Epoch [1773/3000]: Train loss: 1.9620,             Valid loss: 2.6223

Model is not improving, so we halt the training session


# Testing

In [15]:
def save_pred(preds, file):
    '''Save predictions to specified file'''
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

model = My_Model(input_dim=x_train.shape[1]).to(device)
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader, model, device)
save_pred(preds, 'pred.csv')

100%|██████████| 5/5 [00:00<00:00, 198.95it/s]
